# Task 3 - Optimizers

**Requirements:**
 - numpy (https://numpy.org/)
 - matplotlib (https://matplotlib.org/)
or
 - Plotly (https://plotly.com/)

Let's continue with our framework. We use all of the previous implemented classes (with some modifications) and add new - **Optimizers**.

Watch out for the shape of input data.. Now we are working with mini-batches $(B,nX,1)$, where $B$ is number of samples in mini-batch, $nX$ is number of features and $1$ is for vector/matrix multiplication in the last 2 dimensions, leaving $B$ as samples.

In [1]:
# Import
import numpy as np
from utils import Module

In [2]:
#------------------------------------------------------------------------------
#   Linear layer (Dense, Fully connected, Single Layer Perceptron)
#------------------------------------------------------------------------------
class Linear(Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.W = np.random.randn(out_features, in_features)
        self.b = np.zeros((out_features, 1))

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.aPred = input
        self.m = self.aPred.shape[0]
        net = np.matmul(self.W, input) + self.b
        return net

    def backward(self, dz: np.ndarray) -> np.ndarray:
        self.dW = (1.0/self.m) * np.sum(np.matmul(dz, self.aPred.transpose((0,2,1))), axis=0)
        self.db = (1.0/self.m) * np.sum(dz, axis=0)
        return np.matmul(self.W.transpose(), dz)

    def get_optimizer_context(self):
        return [[self.W, self.dW], [self.b, self.db]]

    def update_parameters(self, params):
        self.W, self.b = params
#------------------------------------------------------------------------------
#   SigmoidActivationFunction class
#------------------------------------------------------------------------------
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.fw_input = input
        return 1.0 / (1.0 + np.exp(-input))

    def backward(self, da) -> np.ndarray:
        a = self(self.fw_input)
        return np.multiply(da, np.multiply(a, 1 - a))

#------------------------------------------------------------------------------
#   HyperbolicTangentActivationFunction class
#------------------------------------------------------------------------------
class Tanh(Module):
    def __init__(self):
        super(Tanh, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        self.fw_input = input
        return (np.exp(2 * input) - 1) / (np.exp(2 * input) + 1)

    def backward(self, da) -> np.ndarray:
        a = self(self.fw_input)
        return np.multiply(da, 1 - np.square(a))

#------------------------------------------------------------------------------
#   Model class
#------------------------------------------------------------------------------
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, input) -> np.ndarray:
        for name, module in self.modules.items():
            input = module(input)
        return input

    def backward(self, z: np.ndarray):
        for name, module in reversed(self.modules.items()):
            z = module.backward(z)

## Loss Functions

As in standard deep learning frameworks, calling Loss function can return either **cost** or  **loss**  based on parameter **reduce**.

In [3]:
#------------------------------------------------------------------------------
#   MeanSquareErrorLossFunction class
#------------------------------------------------------------------------------
class MSELoss(Module):
    def __init__(self, reduce="mean"):
        super(MSELoss, self).__init__()
        if reduce == "mean":
            self.reduce_fn = np.mean
        elif reduce == "sum":
            self.reduce_fn = np.sum
        elif reduce is None:
            # return identity (do nothing)
            self.reduce_fn = lambda x : x
        else:
            raise AttributeError

    def forward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return self.reduce_fn(np.mean(np.power(target - input, 2), axis=0, keepdims=True))

    def backward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return np.mean(-2 * (target - input), axis=1, keepdims=True)


#------------------------------------------------------------------------------
#   BinaryCrossEntropyLossFunction class
#------------------------------------------------------------------------------
class BCELoss(Module):
    def __init__(self, reduce="mean"):
        super(BCELoss, self).__init__()
        if reduce == "mean":
            self.reduce_fn = np.mean
        elif reduce == "sum":
            self.reduce_fn = np.sum
        elif reduce is None:
            # return identity (do nothing)
            self.reduce_fn = lambda x : x
        else:
            raise AttributeError

    def forward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return self.reduce_fn(-(target * np.log(input) + np.multiply((1 - target), np.log(1 - input))))

    def backward(self, input: np.ndarray, target: np.ndarray) -> np.ndarray:
        return -np.divide(target, input) + np.divide(1 - target, 1 - input)

## Optimizers

Each optimizer gets as input a **model** and loads each layer's parameters for optimizer context **`layer.get_optimizer_context()`**. Other attributes are based on the optimizer definition. The modified parameters are put back to the model's layer by `layer.set_optimizer_context([W,b])`. Remember that optimizers may require to store some context for the next steps of optimization for each layer and each parameter accordingly.

Your task is to implement:
 - SGD with momentum
 - RMSProp: http://www.cs.toronto.edu/~hinton/coursera/lecture6/lec6.pdf
 - Adam: https://arxiv.org/pdf/1412.6980.pdf

All algorithms are in [https://www.deeplearningbook.org/contents/optimization.html](https://www.deeplearningbook.org/contents/optimization.html)


In [4]:
#------------------------------------------------------------------------------
#   AbstractOptimizer class
#------------------------------------------------------------------------------
class Optimizer:
    def __init__(self):
        pass

    def step(self):
        raise NotImplemented

#------------------------------------------------------------------------------
#   StochasticGradientDescentOptimizer class
#------------------------------------------------------------------------------
class SGD(Optimizer):
    def __init__(self, model:Model, lr:float):
        super(SGD, self).__init__()
        self.model = model
        self.lr = lr
        self.context = {}

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW],[b,db]] = params
                    # >>>> start here
                    W = W - self.lr * dW
                    b = b - self.lr * db
                    # <<<< end here
                    layer.update_parameters([W,b])


In [41]:
#------------------------------------------------------------------------------
#   SGDMomentumOptimizer class
#------------------------------------------------------------------------------
class SGDMomentum(Optimizer):
    def __init__(self, model, lr:float, momentum):
        super(SGDMomentum, self).__init__()
        self.model = model
        self.lr = lr
        self.momentum = momentum
        self.context = {}
        # >>>> start_solution
        # <<<< end_solution

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW],[b,db]] = params
                    if name in self.context.keys():
                    # >>>> start_solution
                    # self.context[name][???] =
                        self.context[name]['vW'] = np.multiply(self.context[name]['vW'], self.momentum) - np.multiply(self.lr, dW)
                        self.context[name]['vb'] = np.multiply(self.context[name]['vb'], self.momentum) - np.multiply(self.lr, db)
                        pass
                    else:
                        self.context[name] = {}
                        self.context[name]['vW'] = np.multiply(-self.lr, dW)
                        self.context[name]['vb'] = np.multiply(-self.lr, db)
                        pass
                    W += self.context[name]['vW'] 
                    b += self.context[name]['vb'] 
                    # <<<< end_solution
                    layer.update_parameters([W,b])

#------------------------------------------------------------------------------
#   RMSpropOptimizer class
#------------------------------------------------------------------------------
class RMSprop(Optimizer):
    def __init__(self, model, lr:float, dec: float, eps=1e-07):
        super(RMSprop, self).__init__()
        self.model = model
        self.context = {}
        # >>>> start_solution
        self.lr = lr
        self.dec = dec
        self.eps = eps
        # <<<< end_solution

    def step(self):
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW], [b, db]] = params
                    if name in self.context.keys():
                    # >>>> start_solution
                    #    self.context[name][???] =
                        self.context[name]['sdW'] = np.multiply((1-self.dec),np.power(dW, 2)) + self.dec*self.context[name]['sdW']
                        self.context[name]['sdB'] = np.multiply((1-self.dec),np.power(db, 2)) + self.dec*self.context[name]['sdB']
                    else:
                        self.context[name] = {}
                        self.context[name]['sdW'] = np.multiply((1-self.dec),np.power(dW, 2))
                        self.context[name]['sdB'] = np.multiply((1-self.dec),np.power(db, 2))
                    # <<<< end_solution

                    W -= self.lr * np.divide(dW, np.sqrt(self.eps + self.context[name]['sdW'])) 
                    b -= self.lr * np.divide(db, np.sqrt(self.eps + self.context[name]['sdB']))

                    layer.update_parameters([W, b])


#------------------------------------------------------------------------------
#   AdamOptimizer class
#------------------------------------------------------------------------------
class Adam(Optimizer):
    def __init__(self, model, lr, dec1, dec2, eps:float = 0.001, delt:float = 1e-08):
        super(Adam, self).__init__()
        self.model = model
        self.context = {}
        # >>>> start_solution
        self.lr = lr
        self.dec1 = dec1
        self.dec2 = dec2
        self.eps = eps
        # <<<< end_solution

    def step(self):
        # >>>>>> Probably add something here ;)
        t = 0
        s = 0
        r = 0
        # <<<<<< until here
        for name, layer in self.model.modules.items():
            if hasattr(layer, 'get_optimizer_context'):
                params = layer.get_optimizer_context()
                if params is not None:
                    [[W, dW], [b, db]] = params
                    if name in self.context.keys():
                        # >>>> start_solution
                        #self.context[name][???] =

                        t += 1
                    else:
                        self.context[name] = {}
                        t += 1
                        self.context[name]['sdW'] = (1-self.dec1) * dW
                        self.context[name]['sdb'] = (1-self.dec1) * db

                        self.context[name]['rdW'] = (1-self.dec2) * np.power(dW,2)
                        self.context[name]['rdb'] = (1-self.dec2) * np.power(db,2) 
                    # <<<< end_solution

                    layer.update_parameters([W, b])



## Main Processing Cell

Watch out for the shape of mini-batch (B,Features,1)

 1. Initialize dataset (`dataset_Flower`).
 2. Declare a simple model.
 3. Initialize optimizer.
 4. Make mini-batches.
 5. Perform forward pass through the network.
 6. Compute loss.
 7. Backward prop loss.
 8. Track loss.
 9. Backward pass MLP.
 10. Use optimizer to modify model parameters.
 11. Repeat for $N$ epochs
 12. Visualize other plots

In [7]:
from utils import gradient_check

In [8]:
from dataset import dataset_Flower, MakeBatches

In [42]:
dataset = MakeBatches(dataset_Flower(m=512, noise=0.3), 32, True)
###>>> start of solution
mlp = Model()
mlp.add_module(Linear(2, 3), 'Dense_1')
mlp.add_module(Tanh(), 'Tanh_1')
mlp.add_module(Linear(3, 4), 'Dense_2')
mlp.add_module(Tanh(), 'Tanh_2')
mlp.add_module(Linear(4, 5), 'Dense_3')
mlp.add_module(Tanh(), 'Tanh_3')
mlp.add_module(Linear(5, 1), 'Dense_4_out')
mlp.add_module(Sigmoid(), 'Sigmoid')
loss_fn = MSELoss(reduce='mean')

#optimizer = SGD(mlp, lr=0.001)
#optimizer = SGDMomentum(mlp, lr=0.001, momentum=0.5)
optimizer = RMSprop(mlp, lr=0.001, dec=0.1)


In [43]:
N_epochs = 1000
losses = []
for i in range(N_epochs):
    if(i%100==0):
        print('Epoch ',i)
    epoch_loss = []
    for mini_batch_X, mini_batch_Y in dataset:
        predicted_Y_hat = mlp.forward(mini_batch_X)
        loss = loss_fn(predicted_Y_hat, mini_batch_Y)
        epoch_loss += [np.mean(loss)]
        dLoss = loss_fn.backward(predicted_Y_hat, mini_batch_Y)
        mlp.backward(dLoss)
        # gradient_check(mlp, loss_fn, mini_batch_X, mini_batch_Y)
        optimizer.step()
    losses += [np.mean(epoch_loss)]

Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900


In [22]:
import plotly.express as px

In [44]:
fig = px.line({'SGD':losses})
fig.show()